In [2]:
!nvidia-smi

Wed Apr  3 07:12:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
#!pip list

In [4]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [5]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.39.3
Uninstalling transformers-4.39.3:
  Successfully uninstalled transformers-4.39.3
Found existing installation: accelerate 0.28.0
Uninstalling accelerate-0.28.0:
  Successfully uninstalled accelerate-0.28.0
  Using cached transformers-4.39.3-py3-none-any.whl (8.8 MB)
  Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)


In [6]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import AdamWeightDecay

In [7]:
# Own Translation System

# Language Translation System

# From the hugging face we are going to use open source Translation model

# Dataset also collected from HuggingFace


In [8]:
model = 'Helsinki-NLP/opus-mt-en-hi'

In [9]:
dataset = load_dataset("cfilt/iitb-english-hindi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
dataset["train"][0]

{'translation': {'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [12]:
tokenizer("Hello my name is sai krishna")

{'input_ids': [12110, 155, 300, 23, 21516, 624, 6984, 896, 5511, 5400, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
#for ex in dataset["train"]["translation"]:
  #print(ex["en"])

In [14]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "hi"

def process_function(sentence):

    inputs = [ex[source_lang] for ex in sentence["translation"]]
    targets = [ex[target_lang] for ex in sentence["translation"]]

    model_inputs = tokenizer(inputs,max_length=max_input_length,truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets,max_length=max_target_length,truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [15]:
tokenized_data = dataset.map(process_function,batched=True)

In [16]:
tokenized_data["train"]["input_ids"][0]

[3872, 85, 2501, 132, 15441, 36398, 0]

In [17]:
model

'Helsinki-NLP/opus-mt-en-hi'

In [18]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [19]:
batch_size = 32
learning_rate = 0.001
weight_decay = 0.01
num_train_epochs = 1

In [20]:
data_collator = DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors="tf")

In [21]:
train_dataset = model.prepare_tf_dataset(
    tokenized_data["test"],
    batch_size = batch_size,
    shuffle = True,
    collate_fn = data_collator
)

In [22]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_data["validation"],
    batch_size = batch_size,
    shuffle = False,
    collate_fn = data_collator
)

In [23]:
optimizer = AdamWeightDecay(learning_rate=learning_rate,weight_decay_rate = weight_decay)

In [24]:
model.compile(optimizer=optimizer)

In [25]:
model.fit(train_dataset,validation_data = validation_dataset,epochs = num_train_epochs)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
78/78 [==============================] - 173s 871ms/step - loss: 4.5738 - val_loss: 4.8313


In [26]:
model.save_pretrained("tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


In [43]:
# Model Inferencing
input_text = "My name is krishna"

In [44]:
tokenized_input = tokenizer([input_text],return_tensors="np")

In [45]:
tokenized_input

{'input_ids': array([[ 633,  300,   23, 6984,  896, 5511, 5400,    0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [46]:
output = model.generate(tokenized_input["input_ids"],max_length=128)

In [47]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(output[0]))

<pad> मेरा नाम तृणमूल है</s> <pad>


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
